### Extracting Resolutions From Pages

Identify:

- resolution dates
- resolution participant lists
- resolution text blocks

In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_dir = os.path.split(os.getcwd())[0]
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

In [51]:
from republic.config.republic_config import base_config, set_config_inventory_num

import copy

inventory_numbers = [
    3767,
    3768,
    3769,
    3783,
    3796,
    3798,
    3819,
    3838,
    3853
]

data_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr"

inventory_num = inventory_numbers[3]
inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)


In [3]:
from republic.fuzzy.fuzzy_context_searcher import FuzzyContextSearcher
from republic.fuzzy.fuzzy_person_name_searcher import FuzzyPersonNameSearcher
from republic.model.republic_phrase_model import resolution_phrases, participant_list_phrases, spelling_variants

from elasticsearch import Elasticsearch
import republic.elastic.republic_elasticsearch as rep_es

es = Elasticsearch()
es_republic = rep_es.initialize_es()

fuzzysearch_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}


fuzzy_searcher = FuzzyContextSearcher(fuzzysearch_config)
fuzzy_person_searcher = FuzzyPersonNameSearcher(fuzzysearch_config)

fuzzy_searcher.index_keywords(resolution_phrases)
fuzzy_searcher.index_spelling_variants(spelling_variants)
#fuzzy_searcher.index_distractor_terms(distractor_terms)



In [6]:
from republic.fuzzy.fuzzy_context_searcher import FuzzyContextSearcher
import republic.elastic.republic_elasticsearch as rep_es

es = Elasticsearch()
es_republic = rep_es.initialize_es()

fuzzysearch_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
    "paragraph_index": "republic_paragraphs",
    "paragraph_doc_type": "paragraph"
}

missing_dates = [
    {"date_string": "Veneris den 5. Januarii 1725.", "page_start": 11, "page_end": 14},
    {"date_string": "Mercuri den 10. Januarii 1725.", "page_start": 21, "page_end": 28},
]

fuzzy_date_searcher = FuzzyContextSearcher(fuzzysearch_config)

for missing_date in missing_dates:
    fuzzy_date_searcher.index_keywords([missing_date["date_string"]])
    for page_num in range(missing_date["page_start"], missing_date["page_end"] + 1):
        paragraphs = rep_es.retrieve_paragraph_by_type_page_number(es_republic, page_num, inventory_config)
        for paragraph in paragraphs:
            if page_num == 25:
                print(paragraph["text"])
            matches = fuzzy_date_searcher.find_candidates(paragraph["text"])
            for match in matches:
                print(match)
                print("page: {}\tDate: {}\tText string: {}\n".format(page_num, match["match_keyword"], match["match_string"]))
                print(paragraph["text"])


NameError: name 'year_config' is not defined

### Indexing Paragraphs with Metadata

In [52]:
import datetime
from elasticsearch import Elasticsearch

import republic.parser.republic_paragraph_parser as para_parser
import republic.elastic.republic_elasticsearch as rep_es
from republic.model.republic_phrase_model import category_index
from republic.config.republic_config import base_config, set_config_year


from republic.config.republic_config import base_config, set_config_inventory_num

import copy

inventory_numbers = [
    3767,
    3768,
    3769,
    3780,
    3783,
    3796,
    3798,
    3819,
    3838,
    3853
]

data_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr"
es = Elasticsearch()
es_republic = rep_es.initialize_es()

inventory_num = inventory_numbers[3]
inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)

#rep_es.delete_es_index(year_config["paragraph_index"])

# start on January first

def initialize_current_date(inventory_config: dict) -> dict:
    year = inventory_config["year"]
    current_date = {
        "month_day": 1,
        "month_name": "Januarii",
        "month": 1,
        "week_day_name": None,
        "year": year
    }
    return current_date
    
#page_docs = rep_es.retrieve_resolution_pages(es_republic, inventory_num, inventory_config)



In [76]:
import republic.parser.republic_paragraph_parser as para_parser
import json

def index_paragraphs(es: Elasticsearch, inventory_config:dict):
    current_date = initialize_current_date(inventory_config)
    page_docs = rep_es.retrieve_resolution_pages(es, inventory_num, inventory_config)
    print("Pages retrieved:", len(page_docs), "\n")
    for page_doc in sorted(page_docs, key = lambda x: x["page_num"]):
        #print("page_id:", page_doc["page_id"], "\ttype:", page_doc["page_type"], "\tnum columns:", len(page_doc["columns"]), "\tnum words:", page_doc["num_words"])
        #break
        paragraphs, header = para_parser.get_resolution_page_paragraphs(page_doc)
        #print("num columns:", len(page_doc["columns"]), "\theader lines:", [line["line_text"] for line in header])
        for paragraph_order, paragraph in enumerate(paragraphs):
            #print(paragraph["text"], "\n\n")
            matches = fuzzy_searcher.find_candidates(paragraph["text"], include_variants=True)
            if para_parser.matches_resolution_phrase(matches):
                paragraph["metadata"]["type"] = "resolution"
            current_date = para_parser.track_meeting_date(paragraph, matches, current_date, inventory_config)
            paragraph["metadata"]["keyword_matches"] = matches
            for match in matches:
                if match["match_keyword"] in category_index:
                    category = category_index[match["match_keyword"]]
                    match["match_category"] = category
                    paragraph["metadata"]["categories"].add(category)
            paragraph["metadata"]["categories"] = list(paragraph["metadata"]["categories"])
            del paragraph["lines"]
            es.index(index=inventory_config["paragraph_index"], doc_type=inventory_config["paragraph_doc_type"], 
                     id=paragraph["metadata"]["paragraph_id"], body=paragraph)



In [77]:
for inventory_num in inventory_numbers:
    print(inventory_num, "\n\n\n")
    if inventory_num < 3583:
        continue
    inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)
    index_paragraphs(es_republic, inventory_config)


3767 



Pages retrieved: 1490 

DAY START: Ben H PR ZE DE, n icere van Nyeveen, c PRESENTIBUS, De Heeren van Randwyck, van Welderen, 
DAY START: Meriurii den 6. Januarii 1712. 14 PRESIDE, Den Heere van Older{om. PRESENTIBUS, De Heerenvan Effen, Huygens, van Broeekhuyfen ‚ Dedem Hars. 
DAY START: Jovis en 7. Januari 1712. SRE SPD Den Heere van Olderfom. SPRESENTIBUS, De Heeren van Efen, Huygens, van Broeckhuyfen ‚ Dedem , Ham. Van Sterrenburgh Repelaer Guldewagen Riyfth, deVicg, Pancras Michtelfz, Mee: |; het Pafport fal moeten werden aengeteecvens, Raedt Penfionaris Heynfins, met een 
DAY START: Veneris den 8. Januari 170. PRESIDE, Den Heere van Olderfom. PRESENTIBUS, De Heeren van Eèn, Huygens ‚ van BroeckDedem Fom Van Sterrenburgh, Repelaer Groenewegen , deVicg, Meerens, Raeds- Penfonaris Heinfins, met een extraardinaris Gedeputeerde uyt de Provincie van Hollandt ende WeftVrieslandt. an Cleverskerche, van Spanbroetk, Coninck, met een extraordinaris Gedeputeerde uyt de  Prowiecie wan

DAY START: Martis den 16. Februari: 171}; PRESIDE, Den Heere Ripperda. PRASENTIBUS, 
DAY START: ij PRE DE, en Heere Ripperda. PRESENTIBUS, De Heeren van Randtwyck, van Welderen van Older[om van Effen, van BroeckhuyJen, Ham. 
DAY START: Sabbathi d aboatht Februari 1712. R ESIDE, Den Heere Ripperda. PR ESENTIEBUS, De Heeren van Randwyck, van Welderen, 
DAY START: Martis den23. Februarij 1713, PRASIDE, Den Heere van Welderen. PRESENTIBUS, 
DAY START: Mercuriì den 24. Februari 1712. PRAESIDE Den Heere van Welderen, LR ESENTIBUS H van Older[om Efe Wij Ham 9 0 je en roeckhuyfen Man Van Sterrenburgh ‚ Repelaer van den Hoert ‚ Graswinckel , de Vicq , Meerens 7 Penfionaris He n[ius 7. kercke, V 7 b k, Coninck Zin vaa EE De toi van Zeeland. P Weede, van Schonauwen 4 
DAY START: L Feb 3 une den 29. Heoruarse 1712. PRESIDE, Den Heere Aepelaer. PRESEN TIEUS, De Heeren van Welderen , van Older[om 3 
DAY START: Martis den 1. Maert 1712. PRASIDE, 
DAY START: PRESENTIBUS, Heeren van Welderen , van Olde

DAY START: Mi id 8 M: ercurn aen 10, Mey 1712, PR&ESIDE, Den Heere van Uterwyck tot Alerdinck. PR&ESENTIBUS, 
DAY START: 5 Jovis den ig. Mey 1712. PRESIDE, Den Heere van Uterwyck tot Alerdinck. PRESENTIBUS De Heeren van Welderen ‚ van Older(om ’ [om , de Beyer, Nuyffenbergh , van Effen , van Broeckhusfen , Ham. Van de Lier ‚ Fagel, van Alphen, Lefteveaon, Raedt - Panfionaris Heinfius met een extraordinaris Gedeputeerde uit de Provincie van Hollandt en We - Vriesland. Van Cleverskercke, Coninck, Van Renssr inde, Du Tour. Sloet, Teliuyden, Roe, Steenbergen, Menfinck. Steenbuyfen, Tamminga, Ripperda: 
DAY START: Veneris den 20. Mey 1713, I Den ee nen Meir, PRESENTIBUS, 
DAY START: Sabbathi den 21. Mey 1712. PRESIDE, Den Heere van Uterwyck tot Alerdinck. PRESENTIBUS, 
DAY START: 4 d Ms Jae AIEN 1712. PRESIDE, Den Heere van Steenhuy[en. PRESENTIBUS, De Heeren van Welderen van Olderfom , de Beyer, Nuyfenbergh, van Broeckhuy[en 7 
DAY START: fl (MI Mercuri den 25, Mey NI IEI LEW 1712. PRESIDE

DAY START: a Veneris den 15. Juli 1712. PR&ESIDE, Den Heere van Ripperda. PRESENTIBUS, 
DAY START: Sabbathi den 16. uli 1712 PRASIDE, Den Heere van Ripperda PRESENTIBUS De Heeren van Broeckhu [en Hom Fagel, van Alphen Raedt Penfi je ie En honaris HeéijA twee extraordinaris Gedeputeerden Jt de Provincie van Hollandt en WefVriesland. Van Cleverskercke ‚ Coninck. Van Renswoude, van Schonauwen, Wan Goflinga , van Burum, da Te Sloet , TJeimuyden our. Een extraordinaris Gedeputeerde uyt de uyt de Provin Cie van Stadt en Lande. 
DAY START: Martis den 26. Juli 1713. PRESIDE, Den Heere Fagel, 
DAY START: PRESENTIBUS Olde Derienen osn Ronde 3 008 Je, van Broeren > E97. Repelaer , van Alphen , van Caftricums NIA Eoleeeane aA Peen Is wee Saarurdinaris Gedeputeerden zit Provincie van Fiollande onde Weft-PrierEIT eerdere, Coninck, met een extraoretis Gedepmteerde uyt Provincie ver Zeeland p y on Vp 4 le TnT: Sl + 7 . ee nn en extraordinatis Gedeputeer” 4 Provincie van Stad en Lande. e unt 4e IV 
DAY

DAY START: »P Vrneris den peen zhe PR&ESIDE, Den Heere van Spanbroeck. PRESEN TIBU S, De Heeren van Older{om, de Beyer, Noy: Senbergh, van Broeckhuyfen Ham, Repelaer ‚ van Alphen , van Caftricum , Les ftevenon, Raedt - Penfionaris Heinfins. Een extraordinaris Gedeputeerde uyt de Provins cie van Zeelandt. Van Weede. Du Tour, Kempenaer. Uterwyck tot Alerdinck, Menfmck. Wichers , Steenhuyfen ‚ Tamminga , Kippers da. 
DAY START: Lune den 26, September 1712. PR&ESIDE Den Heere Port 3 PRESENTIBUS De Heerenvoan Olderfm, de Beyer fenbergh, van Broeckhuyfen Hen; m4 en extraordinaris Gedeputeerde uyt de Provin. cie van Gelderlandt, 
DAY START: Martis den 27. September 1712. PRAESIDE, Den Heere van Weede. PRE.SENSTILITL.S3,; De Heeren van Older[om , de Beyer, Nuy[[enbergh, van Broeckhuyfen, Ham.  Guldewagen , van Alphen van Caftricum , Leftevenon , Egmondt vander Nyenburgh , Raedt - Penfionaris Heinfins. Van Spanbroeck. Du Tour, Kempenaer, met een extraordinaris Gedeputeerde uyt de Provincie van 

...

DAY START: ido 97 Veneris ee Jab 790. PR f S IDE; Den Heere an sider, PRASENTIBUS Bo De Fleeren Een extraordinaris Cedepu- ì eN ‚leerde uit de Lrovmcie van Gelderland. Vanden Sambeuvel Litdh de Jeude, VerOT ij Hollend ef de ee van Holland en efboriesiand. u bil 
DAY START: M. Mercurii den 4 Angufty 1790. PRAESIDE, Den leere Zanden Santheroel. 
DAY START: 1790 (549) Den 5 Autufty l Zoude worden afgewagt het verzoek door den Suppliant te doen. “Dat feedert het voornoemde Meisje Andrefa Charlotta Frederica Neitfch, en ook de Moeder de gemelde vrye Negerin Philis, zynde komen te overlyden, en door dit laatíte Sterfgeval onmogelyk zynde geworden om, zoo al dezelve tot“ de Christelyke Religie was overgezaan, het overgebleeven Kind, door het Huwelyk te leginimeeren, zolks de omftandigheid van dat Kind ‘thans des te fávorabeler was, om dit Beneficie van haar Hoog Mog.'te implorecren; verzoekende den Soppliant derbalven, dat baar Hoog’ Mog. aan den Suppliant tén behoeve van zyn Zoon, den voorno

DAY START: Dane den 33 Setteke 1799. PR&ESIDE, Den Heere Van Spaen, 
DAY START: Den 19 en 13 September PRESEN TIBU 9, De Heeren Verboom, Gevaerts, Raadpgnfionaris vande Spiegel, met een extraor: IN dinaris Gedeputeerde uit ‘de Provincie IN van’ Holland en Weftortesland, A. 3. van Bori]tle, W. Z., van Bor(Jele, van Brakel. IN Bot To Sn en. HA P. U. Rengers, van Groveftins. Hi Fan Suchtelen, U Trip. E Refolutien voorleeden Vrydag genomen, zyn 'geleefen en gerefumeert, gelyk ook gerefumeert wa en gearrefteert zyn de Depeches daar wit rTefuiteerende, Oe Tt IM Go een Miflive van den Heere HI van Lansberge, haar Hoog Mog. Mr °° nifter Plenipotentiaris te Keulen, gefchreeven aldaar den 10 deezer loopende maand, hogdende advertentie, A WAAR op geen refolutie is gevallen, bis 7 
DAY START: Martis den 14 September 179% DR 2-3.1.D:E, Den Heere Zan Spaen. PRESENTIBUS, H Vorden Saksbeucil si: De we age WI SOE narts VanGe Opiegei met EON GAAT io Gedeputeerde uit de Provincie van Holland en MWeftvrie

DAY START: D A S DIE, Nl en Heere Trip. PRA&ASENTIBUS, EN De Heeren Van Welderen, van Rand awvck. van Heeckeren tot Khel! > Cato W. IN eg oee eg Jchalzoord , Calkoen, ‘Twent, Blok, Ie ie DDPge] eG En (| Ei a gh”: IM ene ie HJ KE ie Van van Tuyl van Zuylen, Pefters. il ra le NN eN Nl Vins iet IT din G ae (A ER lt dé WE In oe Voerft tot den pi Im en ait AE Refolutien gifteren B noomen, zvn geleefen en gerefù- EN TE ik es IN gearrefteert zyn de Depeches daar uit refalteerende. 4 Ntfangen een Miflive van den Heers O Gallieris, haar Hoog Mogende Minifter by den Ryksdag ‚ gefchreeven te IM Regensburg den 21 deezer loopende maand , geaddrefleert aan den Griffier Fagel, dende advertentie. WAAR op geen refolutie is gevallen le GERE 
DAY START: Veneris den 29 Oftober 1790. PIR. D..E, Den Heere Trip. PRESENTIBUS, 
DAY START: ji Et ll Dine Ve ee || 9% EE Nibil a&um eft. 1 Lane den 1 November 1790. Te GECOMPBARFERI Zyne Hoogheid den Heere Prince van HE Orange en Na[]an. P R&S 1 DE, Den. Heefe ‘Van 

DAY START: Pp 7 RESENTIBU 93) 
DAY START: Den 30 November (798) 1790 Martis den 30 November 1790. PRAESIDE Den: Heere Shri HDE, PRA&SENTIBUS, De Heeren Van Welderen. Van Walfenacr Starrenburg, van Slingelandt ‘van Goidfchalsoord, Raadpen/ionaris vande Spiegel. Van Citters, A. J. van Bor(ele, W. Z. van Bor(]ele, van Brakell. Van Heeckeren van Brantfenburg, Defters. Van Aylva, P. U. Rengers, van Schuy: lenburch, van Groveftins. Peftrinck , Roufe. Trip, 
DAY START: ov is den 2 December 1790. p Hd RE SIDE, Den Heere Van Scheltinga. PRESENTIBUS DeiHoerenVen Welderen > Fan WajTenaer Catwyck ga Wafen Soeren Dt ee enaer Slingelandt van GoidUNE EN Feude, Calkoen, vanaer Durt went, Moffel, Ver ‚ Moljel, Ver: boom, Raadpen/ionaris vande Sri 3 vande Spiegel. Van EET A. FJ. van Bor(tle, van bell. 
DAY START: Veneris den 3 December 1700, 79 PR&ESIDE, Den Heere Van Scheltinga. PRESENTIBUS De Heeren Van Welderen. Van Walenaer Catwyck, Waljenaer Starrenburg , van Slingelandt van Goid-. febalsoord, Lidh

DAY START: Martis den 28 December 1790. PRAÂ SIDE, Den Heere Van Ma{Jenacr Catwyck. EPRESENTIBUS, 
DAY START: Mercuri den 29 December 1790. PR A4 531D'E, HN . Den Heere Jan Jaffenaer Catwyck. PRESENTIBU S, De Heeren Van Welderen, van Heeckeren ll tat Nettelborft, van Heeckeren tot Wiefseee en de uis de Provincie van Gelderland. Van Slingelandt van Goidfchalsoord ‚ LitdB Tl de 
DAY START: Jovis den 30 December 1799. PRASIDE, Den Heere Zan Wa{enaer Catwyck. PRASENTIBUS, De Heeren Van Welderen, van Heecke. ren tot Nettelhorft eig Hedcktzen tt Wieren, met 46 ORDE Gede. puteerde it de: Provincie van G ld td GEGE Van Slingelandt van Coidfchaloord , Lied de Feude vander Burch. Toe M fel Jerhoon Raet ont > Spiegel Att en ee vi ee teert de: Provincie nie We[turiesland ge Van Cites pn F.van Bor(Jel Van Heeckeren Tt enen Teli van: Zuvien Fo erg WE, om Van ea B P UR Vs Roue Wichers 
DAY START: 1790 (905) Veneris fa Derember 199 PR&ESIDE, Den Heere Van Wafenaer Catwyck. PRASENTIBUS, De Heeren Van W

In [62]:
inventory_num = 3783
inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)
#index_paragraphs(es, inventory_config)

current_date = initialize_current_date(inventory_config)
page_docs = rep_es.retrieve_resolution_pages(es, inventory_num, inventory_config)
print("Pages retrieved:", len(page_docs), "\n")



Pages retrieved: 966 



In [74]:
for page_doc in sorted(page_docs, key = lambda x: x["page_num"]):
    if page_doc["page_num"] < 131 or page_doc["page_num"] > 132:
        continue
    print(page_doc["page_num"])
    for column in page_doc["columns"]:
        for line in column["lines"]:
            print("({}, {}) ({}, {})".format(line["left"], line["right"], line["top"], line["bottom"]), line["line_text"])
    paragraphs, header = para_parser.get_resolution_page_paragraphs(page_doc)
    for paragraph in paragraphs:
        print(paragraph["text"])


131
(429, 2230) (331, 398) Den 14. Januari (30) 1728.
(597, 2213) (438, 471) Heere van Renswoude ter Verga- haar getrocken zyn of gepreetendeert wor-
(547, 2212) (492, 524) deringe , heeft aan haar den; dat daar op aan haar Hoogh Mogen-
(548, 2213) (545, 576) Hoogh Mogende voorgedraagen ende / de moeft berichten , dat voor omtrent {e-
(430, 2215) (578, 621) bekent gemaackt, dat den Heere Grave van {tien maanden geleden geworden zynde Drof-
(432, 2213) (634, 668) Konigiegg-Erps, extraordinaris Envoyé van {aardt der voornoemde Heerlijckheydt van
(432, 2211) (674, 713) fijne Keyterlijcke Majefteyt, by hem was Moergeftel, aldaar ‘hadde ondervonden, dat
(431, 2214) (738, 760) geweeft , ende een conferentie met haar Johan van Vaerle , gewefen Droflaardt tot
(431, 2213) (771, 809) Hoogh Mogende Gedeputeerden hadde ver- Moergettel, aldaar hadde geftoort den Room-
(417, 2214) (835, 860) focht. WAAR op gedelibereert {chen Godtsdientt , doch dat eyndelijck in
(433, 2215) (874, 910) zynde, is goed

## Test REPUBLIC hOCR Model

In [57]:
inventory_num = 3769
inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)
current_date = initialize_current_date(inventory_config)
page_docs = rep_es.retrieve_resolution_pages(es, inventory_num, inventory_config)


In [61]:
import json
from republic.model.republic_hocr_model import HOCRObject, HOCRColumn, HOCRPage

page_doc = page_docs[0]
print(page_doc["page_id"])
print(page_doc["num_words"])
for column in page_doc["columns"]:
    for line in column["lines"]:
        print(line["line_text"])
#print(json.dumps(page_doc, indent=2))

hocr_columns = [HOCRColumn(column, inventory_config) for column in page_doc["columns"]]
hocr_page = HOCRPage(page_doc, inventory_config)
for paragraph in para_parser.get_resolution_paragraphs(hocr_page, inventory_config):
    print(paragraph.text)
    print()

year-1714-scan-666-odd
633
oen. ier ter V
ergaderin
e I2
p ter Vergad 5e te 76)
van Coil gaderinge
een XR a Lambert gelcfen d
v
Aeg e iR
Olien eg Colon van van je EPs 1 den as
van (ij ‚ be n
ter van Regim ijn Resi nevens d vyftie
foeck Geuthe ent van d giment ce ve
ende m, nu 1 cn Coil ende rtentie er loope d en 1e W er Hof
queft om r aetÍtelij lonel Pi val!
gee gegen» me in Na, ei
o
o t u
p gedeli gen we peufioe reve
ende jj z rien, DEN gebenefi-
{ch rÍtaen ynde WA
dr Requcfte ij Copie Vran dinaris her Boogh) en den H
an d efteld van d en anckrijck
de and e Heeren t {alwe ev twinti
oor-
teerden ere haer H van ek en JN ha geaddr
ende tot de ENT Mo uyfen daer clieert aen
den es deni itaire {a gende Ged en- tenti
yt den nige H eN e=
te nomin Raedt va Geren Ged om me vallen
van all eren, te Vi n State, b commits
d
es alhi reren y hae er-
97m. ier ter a felfs O7
gaderinge Ta En 9
Marti ee
rt aend
den 25. D geen t, houdend vw ncightten tet
1714 ecember
ads ne Ox
Nihil ersdagh
Mercurii mae
u